In [ ]:
#FACIAL EMOTION RECOGNITION
#importing the required modules
import sys, os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
import tensorflow as tf
#reading the dataset.This is the dataset which is available in internet.This is a csv file with columns "emotion,pixels,usage"
dataset=pd.read_csv('./fer2013.csv')
#array variables for storing training data for x axis and y axis ,testing data for x axis and y axis 
xtrain,ytrain,xtest,ytest=[],[],[],[]
#seperating training data,private testing data and public testing data
#looping through out the csv file and assigning values to particular arrays
for index, row in dataset.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           xtrain.append(np.array(val,'float32'))
           ytrain.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           xtest.append(np.array(val,'float32'))
           ytest.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

#variables for number of features,number of labels,batch size i.e,how many parts the data set is divided.
#number of features
no_features = 64
#number of labels i.e,no of emotions=happy,sad,angry,fear,surprized,disgust,neutral.
no_lbls = 7
#batch size=into how many parts the data is divided.
bth_size = 64
#epochs =the repetitive number of times the data is trainined by the model.
no_epochs = 2
#width and height of the image i.e,image is vertically divided into 48 pixels and horizontally divided into 48 pixels
width, height = 48, 48

#coverting normal arrays to one hot vectors so that the convolutional neural networks accept them.
xtrain = np.array(xtrain,'float32')
ytrain = np.array(ytrain,'float32')
xtest = np.array(xtest,'float32')
ytest = np.array(ytest,'float32')

ytrain=np_utils.to_categorical(ytrain, num_classes=no_lbls)
ytest=np_utils.to_categorical(ytest, num_classes=no_lbls)

#normalizing the training data so that the training data is placed between 'zero' and 'one'
xtrain -= np.mean(xtrain, axis=0)
xtrain /= np.std(xtrain, axis=0)
#normalizing the testing data so that the testing data lies between zero and one
xtest -= np.mean(xtest, axis=0)
xtest /= np.std(xtest, axis=0)
#reshaping the data so that it is given as 2D input to the convolutional layers.
xtrain = xtrain.reshape(xtrain.shape[0], 48, 48, 1)

xtest = xtest.reshape(xtest.shape[0], 48, 48, 1)
#designing a 'CONVOLUTIONAL NEURAL NETWORK' for training the model.
model = Sequential()
#adding 2 convolutional layers simultaneously and followed by a max pooling layer.
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',padding='same',input_shape=(xtrain.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
#dropout is added to the model so that the model we are training doesnot undergo overfitting.
model.add(Dropout(0.5))
#adding the next 2 convolutional layers and a maxpooling layer
model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#adding the last 2 convolutional layers and a maxpooling layer.
model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
#flattening the 2D convolutional layers into 1D fully connected layers.
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
#using the softmax so that the output of each label is produced as percentange.
model.add(Dense(no_lbls, activation='softmax'))
#creating a check point so that the best accurated training is taken into count whereas the least accurated one is ignored
checkpoint = tf.keras.callbacks.ModelCheckpoint('fer.h5',
                            monitor='loss',
                            verbose=1,
                            save_best_only=True,
                            mode='auto')

#model is compiled and the accuracy metric is viewed.
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

#fitting the dataset to the model.i.e, training the model.
model.fit(xtrain, ytrain,
          batch_size=bth_size,
          epochs=no_epochs,
          verbose=1,
          validation_data=(xtest, ytest),
          shuffle=True)


#saving the model as a json file so that we can reuse the model.
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")

Epoch 1/2
435/449 [============================>.] - ETA: 47s - loss: 1.6756 - accuracy: 0.3295